In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

In [31]:
reviews = pd.read_csv('./Amsterdam/reviews2.csv')
listings_desc = pd.read_csv('./Amsterdam/listings2.csv')
listings = pd.read_csv('./Amsterdam/listings.csv')
neighbour = pd.read_csv('./Amsterdam/neighbourhoods.csv')
calendar = pd.read_csv('./Amsterdam/calendar.csv')

#C:\Users\avadi\Desktop\DataScience_Udacity\Project 1\Amsterdam

In [3]:
listings.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,816940,Old South Lovely Garden Apartment,4290554,Joyce,NaN,Zuid,52.35323,4.85421,Entire home/apt,165,6,63,2022-11-28,0.54,1,11,2,03632A9BE56A736890D1
1,528022,Somewhere Else,2594559,Els,NaN,Westerpark,52.39114,4.88846,Private room,100,2,248,2022-11-28,3.30,1,212,37,0363 9289 A94D 5C21 A579
2,538723,Beautiful ground floor apartment,356740,Dimphy,NaN,Watergraafsmeer,52.35400,4.93965,Entire home/apt,140,5,57,2020-01-01,0.45,1,0,0,0363 D990 F92F DECF 88D4
3,549310,Family Apartment with Garden,2699033,Barbara,NaN,De Baarsjes - Oud-West,52.37126,4.86241,Entire home/apt,200,2,54,2022-08-10,0.60,1,7,7,0363 35CD 76C9 04D4 1A21
4,553514,"B&B WestergasTverblijf , Cosy!",618589,Mirjam,NaN,Westerpark,52.38444,4.86916,Entire home/apt,80,2,162,2020-02-16,1.34,1,1,0,0363 AD82 ADCF 17F3 C3EA


## Pre-process Data 

### Calendar Data

In [4]:
# Looking for nulls
calendar.isnull().sum()

# Looking for listings where price is null
calendar[calendar.price.isnull()].listing_id.unique()


array([          12550549,           32234160,           37591658,
                 37614789, 720266014528319438,           18631508],
      dtype=int64)

In [5]:
# It is safe to drop all rows that have null values as there are very little days/listings where the price is missing
calendar_clean = calendar.dropna()

In [6]:
# Number listings in/around Amsterdam = 6809
len(calendar_clean.dropna().listing_id.unique().tolist())

6809

In [7]:
calendar_clean['available']  = np.where(calendar_clean['available']=='t' , 1, 0)
calendar_clean['year'] = pd.to_datetime(calendar_clean['date']).dt.year
calendar_clean['month'] = pd.to_datetime(calendar_clean['date']).dt.month
calendar_clean['day'] = pd.to_datetime(calendar_clean['date']).dt.day_name()

<ipython-input-7-58eccb52175d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar_clean['available']  = np.where(calendar_clean['available']=='t' , 1, 0)
<ipython-input-7-58eccb52175d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar_clean['year'] = pd.to_datetime(calendar_clean['date']).dt.year
<ipython-input-7-58eccb52175d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [8]:
x = ['listing_id', 'year', 'month', 'day', 'minimum_nights','maximum_nights']
calendar_clean[["price", "adjusted_price"]] = calendar_clean[["price", "adjusted_price"]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(pd.to_numeric)
calendar_clean = calendar_clean.groupby(x, as_index = False, group_keys = False).aggregate({'available':'sum','price':['mean'],'adjusted_price':'mean'})
calendar_clean.columns = calendar_clean.columns.droplevel(1)

<ipython-input-8-fd4015009f53>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  calendar_clean[["price", "adjusted_price"]] = calendar_clean[["price", "adjusted_price"]].apply(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).apply(pd.to_numeric)
c:\Users\avadi\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [9]:
calendar_clean.head(30)

,listing_id,year,month,day,minimum_nights,maximum_nights,available,price,adjusted_price
0,2818,2022,12,Friday,3,1125,0,69.00,69.00
1,2818,2022,12,Monday,3,1125,1,59.00,59.00
2,2818,2022,12,Saturday,3,1125,0,69.25,69.25
3,2818,2022,12,Sunday,3,1125,0,59.00,59.00
4,2818,2022,12,Thursday,3,1125,0,69.00,69.00
5,2818,2022,12,Tuesday,3,1125,2,59.00,59.00
6,2818,2022,12,Wednesday,3,1125,2,69.00,69.00
7,2818,2023,1,Friday,3,1125,3,59.00,59.00
8,2818,2023,1,Monday,3,1125,3,67.00,67.00
9,2818,2023,1,Saturday,3,1125,3,59.00,59.00


### Listings Data

In [59]:
listings.isnull().sum()
listings_clean = listings.drop(['license', 'neighbourhood_group', 'number_of_reviews_ltm',
                                 'last_review', 'price', 'host_id','host_name'], axis=1)
sum(listings_clean.reviews_per_month.isnull() == (listings_clean['number_of_reviews'] == 0))

6809

In [41]:
listings_clean.isnull().sum()
listings_clean['reviews_per_month'] = listings_clean['reviews_per_month'].fillna(0)

In [42]:
listings_clean

,id,name,neighbourhood,room_type,number_of_reviews,reviews_per_month,calculated_host_listings_count
0,816940,Old South Lovely Garden Apartment,Zuid,Entire home/apt,63,0.54,1
1,528022,Somewhere Else,Westerpark,Private room,248,3.30,1
2,538723,Beautiful ground floor apartment,Watergraafsmeer,Entire home/apt,57,0.45,1
3,549310,Family Apartment with Garden,De Baarsjes - Oud-West,Entire home/apt,54,0.60,1
4,553514,"B&B WestergasTverblijf , Cosy!",Westerpark,Entire home/apt,162,1.34,1
...,...,...,...,...,...,...,...
6804,733683952767062833,Trendy and cozy apartment with garden,De Baarsjes - Oud-West,Entire home/apt,2,1.54,1
6805,733803741061691698,Mooi appartement met balkon in Amsterdam Oost,Oostelijk Havengebied - Indische Buurt,Entire home/apt,0,0.00,1
6806,733852793472777240,"Nette, mooi woning in Amsterdam.",Centrum-Oost,Entire home/apt,0,0.00,1
6807,733866906581223578,Luxe appartement aan de gracht,Centrum-West,Entire home/apt,0,0.00,1


In [56]:
listings_desc_clean = listings_desc.drop(['listing_url','scrape_id','last_scraped','source',
                                          'name','description','neighborhood_overview',
                                          'picture_url','host_id', 'host_url','host_name',
                                          'host_location','host_about', 'host_thumbnail_url',
                                          'host_picture_url','host_neighbourhood','host_listings_count',
                                          'host_verifications','host_has_profile_pic','host_identity_verified',
                                          'license', 'neighbourhood','neighbourhood_cleansed',
                                          'neighbourhood_group_cleansed','latitude','longitude', 
                                          'calculated_host_listings_count',
                                          'calculated_host_listings_count_entire_homes',
                                          'calculated_host_listings_count_private_rooms',
                                          'calculated_host_listings_count_shared_rooms',
                                          'minimum_minimum_nights', 'maximum_minimum_nights',
                                          'minimum_maximum_nights', 'maximum_maximum_nights',
                                          'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
                                          'minimum_nights', 'maximum_nights','has_availability', 
                                          'availability_30', 'availability_60', 'availability_90',
                                          'calendar_last_scraped','bathrooms','first_review', 'last_review',
                                          'number_of_reviews', 'number_of_reviews_ltm','number_of_reviews_l30d','host_total_listings_count'], axis=1)



listings_desc_clean.head(20)

,id,host_since,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,property_type,room_type,accommodates,bathrooms_text,...,availability_365,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,reviews_per_month
0,528022,2012-06-10,within a day,87%,96%,f,Private room in boat,Private room,2,1.5 baths,...,212,4.85,4.88,4.83,4.93,4.90,4.78,4.72,f,3.30
1,2818,2008-09-24,within an hour,100%,100%,t,Private room in rental unit,Private room,2,1.5 shared baths,...,27,4.89,4.93,4.99,4.97,4.98,4.69,4.81,f,1.88
2,20168,2009-12-02,within an hour,100%,98%,f,Private room in townhouse,Private room,2,1 private bath,...,0,4.44,4.69,4.79,4.63,4.62,4.87,4.49,t,2.18
3,27886,2010-03-23,within an hour,100%,100%,t,Private room in houseboat,Private room,2,1.5 baths,...,40,4.94,4.93,4.95,4.95,4.92,4.89,4.79,t,1.83
4,28871,2010-05-13,within a few hours,100%,100%,t,Private room in rental unit,Private room,2,1 shared bath,...,91,4.88,4.93,4.89,4.95,4.94,4.97,4.83,f,3.03
5,29051,2010-05-13,within a few hours,100%,100%,t,Private room in rental unit,Private room,1,1 shared bath,...,142,4.79,4.88,4.85,4.91,4.91,4.88,4.77,f,4.22
6,44391,2010-08-08,NaN,NaN,100%,f,Entire rental unit,Entire home/apt,4,1.5 baths,...,0,4.72,4.68,4.45,4.95,4.90,4.68,4.50,f,0.30
7,49552,2010-09-06,within a few hours,100%,92%,t,Entire guest suite,Entire home/apt,3,1 bath,...,175,4.92,4.92,4.91,4.95,4.97,4.98,4.78,f,3.06
8,50523,2010-09-12,within an hour,100%,99%,t,Private room in bed and breakfast,Private room,2,1 private bath,...,211,4.87,4.90,4.88,4.87,4.85,4.94,4.83,f,2.50
9,55868,2010-10-17,within an hour,100%,25%,f,Entire rental unit,Entire home/apt,4,1.5 baths,...,31,4.86,4.91,4.86,4.95,4.98,4.95,4.80,f,0.65


In [13]:
sum(listings.reviews_per_month.isnull() == (listings['number_of_reviews'] == 0))

6809

In [38]:
#calendar.join(listings, index='listing_id', lsuffix="_left", rsuffix="_right")

#pd.merge(calendar_clean, listings, on=['listing_id','id'])

x = (pd.merge(calendar_clean, listings_clean, left_on='listing_id', right_on='id'))
#x_2818 = 
#sum(x[x['listing_id'] == 2818]['available'])
#listings[listings['id'] == 2818]


array(['Oostelijk Havengebied - Indische Buurt', 'Centrum-Oost',
       'Centrum-West', 'Zuid', 'Oud-Oost', 'De Pijp - Rivierenbuurt',
       'Slotervaart', 'De Baarsjes - Oud-West', 'Bos en Lommer',
       'Westerpark', 'IJburg - Zeeburgereiland', 'Watergraafsmeer',
       'Noord-Oost', 'Buitenveldert - Zuidas', 'Oud-Noord', 'Noord-West',
       'Geuzenveld - Slotermeer', 'De Aker - Nieuw Sloten', 'Osdorp',
       'Bijlmer-Centrum', 'Gaasperdam - Driemond', 'Bijlmer-Oost'],
      dtype=object)

In [15]:
calendar[calendar.listing_id == 2818]

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,2818,2022-12-05,f,$59.00,$59.00,3,1125
1,2818,2022-12-06,t,$59.00,$59.00,3,1125
2,2818,2022-12-07,t,$59.00,$59.00,3,1125
3,2818,2022-12-08,f,$59.00,$59.00,3,1125
4,2818,2022-12-09,f,$59.00,$59.00,3,1125
...,...,...,...,...,...,...,...
360,2818,2023-11-30,f,$75.00,$75.00,3,1125
361,2818,2023-12-01,f,$76.00,$76.00,3,1125
362,2818,2023-12-02,f,$76.00,$76.00,3,1125
363,2818,2023-12-03,f,$76.00,$76.00,3,1125
